In [1]:
from modules import *
from tokeniser import *
from process_blog_csv import *

In [7]:
blogs = get_blogs(1000)

In [8]:
tokeniser = Subword_Tokeniser()
tokeniser.fit_on_text_BPE(blogs[:100],300)

In [9]:
window_size = 256
vocab_size = tokeniser.vocab
embedding_height = 128

In [10]:
x,y = blogs_to_train(tokeniser,blogs,100000,window_size)

In [11]:
model = tf.keras.Sequential()
model.add(Embedding(vocab_size,embedding_height,mask_zero = True,input_length = window_size))
model.add(Positional_Encode())
model.add(Transformer_Block(embedding_height,window_size = window_size))
model.add(Transformer_Block(embedding_height,window_size = window_size))
model.add(Transformer_Block(embedding_height,window_size = window_size))
model.add(Dense(vocab_size,activation = "softmax"))

model.compile(loss = "sparse_categorical_crossentropy",optimizer = "adam",metrics = ["accuracy"])

In [12]:
model.fit(np.array(x).astype("float32"),np.array(y).astype("int32"))

3125/3125 [==============================] - 6717s 2s/step - loss: 3.7125 - accuracy: 0.1866


In [13]:
model.fit(np.array(x).astype("float32"),np.array(y).astype("int32"),epochs = 5)

Epoch 1/5
3125/3125 [==============================] - 6249s 2s/step - loss: 3.0468 - accuracy: 0.2958
Epoch 2/5
3125/3125 [==============================] - 6252s 2s/step - loss: 2.6022 - accuracy: 0.3875
Epoch 3/5
3125/3125 [==============================] - 6235s 2s/step - loss: 2.3293 - accuracy: 0.4463
Epoch 4/5
3125/3125 [==============================] - 6263s 2s/step - loss: 2.1436 - accuracy: 0.4866
Epoch 5/5
3125/3125 [==============================] - 7010s 2s/step - loss: 2.0112 - accuracy: 0.5161


In [14]:
model.save("gbt-256x128")

INFO:tensorflow:Assets written to: gbt-256x128\assets


INFO:tensorflow:Assets written to: gbt-256x128\assets


In [20]:
def generate(input_sentence,new_tokens):
    encoded_text = tokeniser.encode([input_sentence])[0][:-1]

    for i in range(new_tokens):
        padding = [0 for _ in range(window_size - len(encoded_text))]
        padding.extend(encoded_text)
        inputs = np.array(padding).reshape(1,window_size)

        outputs = model.predict(inputs,verbose = False)[0]
        outputs[-1][np.argmax(outputs[-1])] = 0
        encoded_text.append(np.argmax(outputs[-1]))

        print(tokeniser.decode([encoded_text])[0],end = "\r")

    return tokeniser.decode([encoded_text])

generate("I think I am going ",100)

["I think I am going... leeat these boatched these are you! If why, oily constantsot, Li did! Nichollected that amounty belbitter that there to be fter was so many sing..n'd for the ghed ch her heads thing seem son,&BU2 dride at the gintroke a nutes to be ht thice. 62 d (as "]